<a href="https://colab.research.google.com/github/espickle1/boltz-2/blob/main/boltz_yaml_creator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boltz[cuda] -U

In [ ]:
# === Interactive YAML builder (ONLY id + sequence) ===
# Prompts for YAML directory and output filename
!pip -q install ipywidgets
from google.colab import output, files
output.enable_custom_widget_manager()

import ipywidgets as W
from IPython.display import display, Markdown, clear_output
import os

# ----- protein entry factory: ONLY 'id' and 'sequence' -----
def make_entry():
    id_field = W.Text(placeholder="protein id", layout=W.Layout(width="350px"))
    seq_field = W.Textarea(placeholder="protein sequence", layout=W.Layout(width="600px", height="80px"))
    row = W.VBox([
        W.HBox([W.Label("id:", layout=W.Layout(width="30px")), id_field]),
        W.HBox([W.Label("sequence:", layout=W.Layout(width="70px")), seq_field]),
        W.HTML("<hr>")
    ])
    return {"box": row, "id": id_field, "seq": seq_field}

# container for multiple entries
entries = []

def add_entry(_=None):
    e = make_entry()
    entries.append(e)
    entries_box.children = [x["box"] for x in entries]

def remove_last(_=None):
    if entries:
        entries.pop()
        entries_box.children = [x["box"] for x in entries]

# initial one
entries_box = W.VBox([])
add_btn = W.Button(description="Add protein", button_style="info")
remove_btn = W.Button(description="Remove last", button_style="")
preview_btn = W.Button(description="Preview YAML", button_style="primary")
save_btn = W.Button(description="Save YAML", button_style="success")
out = W.Output()

# Directory and filename widgets
dir_text = W.Text(value="/content/", description="Directory:", layout=W.Layout(width="500px"))
file_text = W.Text(value="protein.yaml", description="File name:", layout=W.Layout(width="300px"))

add_btn.on_click(add_entry)
remove_btn.on_click(remove_last)

# ---- YAML (manual, tabs) ----
def build_yaml_text():
    parts = ["sequences:"]
    for e in entries:
        pid = e["id"].value.strip()
        pseq = e["seq"].value.strip()
        if not pid and not pseq:
            continue
        parts.append("\t- protein:")
        parts.append(f"\t\t\tid: {pid}")
        parts.append(f"\t\t\tsequence: {pseq}")
    return "\n".join(parts) + ("\n" if parts else "")

def on_preview(_):
    with out:
        clear_output()
        y = build_yaml_text()
        if not entries:
            display(Markdown("**Add at least one protein entry.**"))
            return
        display(Markdown("### YAML Preview"))
        display(Markdown(f"```\n{y}```"))

def on_save(_):
    with out:
        clear_output()
        y = build_yaml_text()
        if not entries:
            display(Markdown("**Add at least one protein entry.**"))
            return
        # Get path and filename
        directory = dir_text.value.strip()
        fname = file_text.value.strip()
        if not (fname.endswith(".yaml") or fname.endswith(".yml")):
            fname += ".yaml"
        # Ensure directory exists
        os.makedirs(directory, exist_ok=True)
        full_path = os.path.join(directory, fname)
        with open(full_path, "w", encoding="utf-8") as f:
            f.write(y)
        display(Markdown(f"**Saved:** `{full_path}`"))
        # Also offer download
        files.download(full_path)

preview_btn.on_click(on_preview)
save_btn.on_click(on_save)

# layout
controls = W.HBox([add_btn, remove_btn, preview_btn, save_btn])
display(W.VBox([
    W.HTML("<h3>YAML: sequences → - protein → (id, sequence)</h3>"),
    dir_text,
    file_text,
    controls,
    entries_box,
    out
]))

# add one empty entry by default
add_entry()